# Joint PDf of $\delta_{R_1}(r)$, $\delta_{R_2}(r+s)$

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import scipy
from scipy.stats import multivariate_normal
import seaborn

from densitysplit import DensitySplit
from densitysplit.lognormal_model import *
from densitysplit.utils import confidence_ellipse

# to automatically reload imported modules when executing cell
%load_ext autoreload
%autoreload 2

plt.style.use(os.path.join(os.path.abspath(''), 'densitysplit.mplstyle'))

plots_dir = '/feynman/home/dphp/mp270220/plots/density'

2024-06-19 11:15:16.700140: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:67] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/rhel-8.x86_64/gnu9/hdf5/1.10.8/lib:/opt/rhel-8.x86_64/modulefiles/gsl/26/lib:/datablock:/feynman/work/dphp/adematti/cosmodesiconda/20230918-1.0.0/conda/lib:/feynman/work/dphp/adematti/cosmo/cosmodesiconda/20230918-1.0.0/common/planck/code/plc_3.0/plc-3.1/lib:/feynman/work/dphp/adematti/cosmodesiconda/20230918-1.0.0/aux/lib
2024-06-19 11:15:16.700180: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:264] failed call to cuInit: UNKNOWN ERROR (303)
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


### Load DensitySplit

In [2]:
# Mock parameters
z = 0.8
nbar = 0.0034
rsd = True

# Density smoothing parameters
cellsize = 10
cellsize2 = 10
resampler = 'tsc'

ds_dir = '/feynman/work/dphp/mp270220/outputs/densitysplit/'
simname = 'AbacusSummit_2Gpc_z{:.3f}_ph0{{:02d}}_downsampled_particles_nbar{:.4f}'.format(z, nbar)
cs2 = '_cellsize{:d}'.format(cellsize2) if cellsize2 is not None else ''
densityname = simname.format(0)+'_cellsize{:d}{}_resampler{}_delta_R{}'.format(cellsize, cs2, resampler, '_RSD' if rsd else '')

ds_fn = simname+'_cellsize{:d}{}_resampler{}_3splits_randoms_size4_RH_CCF{}'.format(cellsize, cs2, resampler, '_RSD' if rsd else '')
dsplit = DensitySplit.load(os.path.join(ds_dir, ds_fn.format(0)+'.npy'))

### Plot 2D PDF of $\delta_{R_1}$, $\delta_{R_2}$

In [3]:
def scatter_hist(x, y, ax, ax_histx, ax_histy, bins=100, label1=None, label2=None, size=3, legend=None, sigma1=None, sigma2=None):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y, s=size, label=legend, alpha=0.1)
    seaborn.kdeplot(x=x, y=y, ax=ax, levels=[0.125, 0.61], color='blue', alpha=0.7)
    ax.set_xlabel(label1)
    ax.set_ylabel(label2)
    ax.legend(loc='upper right')

    delta = np.linspace(-2, 2, 100)
    ax_histx.hist(x, bins=bins, alpha=0.7, density=True)
    ax_histx.plot(delta, scipy.stats.norm.pdf(delta, 0, sigma1), color='C1')
    ax_histy.hist(y, bins=bins, alpha=0.7, density=True, orientation='horizontal')
    ax_histy.plot(scipy.stats.norm.pdf(delta, 0, sigma2), delta, color='C1')

In [ ]:
r1 = 10
r2 = 10
sep = 50
mu = 0 if rsd else None
split = 1
size = 10000
los = 'x'

deltaR1, deltaR2 = dsplit.compute_jointpdf_delta_R1_R2(s=sep, sample_size=size, mu=mu, los=los, split=split)

# lognormal model
model = LognormalDensityModel()
sigma1, delta01 = model.get_params_from_moments(sample=deltaR1)
sigma2, delta02 = model.get_params_from_moments(sample=deltaR2)
# transform variables to get Gaussian distributions
x = np.log(1 + deltaR1/delta01) + sigma1**2/2.
y = np.log(1 + deltaR2/delta02) + sigma2**2/2.

# Start with a square Figure.
fig = plt.figure(figsize=(5, 5))
gs = fig.add_gridspec(2, 2,  width_ratios=(3, 1), height_ratios=(1, 3),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)

# Create the Axes.
ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

label1=r'$\ln \left[ 1 + \delta(R_1 = {}\; \mathrm{{Mpc}}/h) / \delta_{{0, 1}} \right]$'.format(r1)
label2=r'$\ln \left[ 1 + \delta(R_2 = {}\; \mathrm{{Mpc}}/h) / \delta_{{0, 2}} \right]$'.format(r2)
legend=(r'$s = {} \; \mathrm{{Mpc}}/h, \; \mu = {}$'.format(sep, mu) if rsd else r'$s = {} \; \mathrm{{Mpc}}/h$'.format(sep)) + "\n" + ('DS{}'.format(split) if split is not None else '')
# Draw the scatter plot and marginals.
scatter_hist(x, y, ax, ax_histx, ax_histy, label1=label1, label2=label2, legend=legend, sigma1=sigma1, sigma2=sigma2)
confidence_ellipse(x, y, ax, n_std=1.0, edgecolor='C1', ls='--')
confidence_ellipse(x, y, ax, n_std=2.0, edgecolor='C1', ls='--')

plt.savefig(os.path.join(plots_dir, 'joint_density_PDF_r{}_r{}_s{}_mu{}{}{}.png'.format(r1, r2, sep, mu, '_RSD' if rsd else '', '_DS{}'.format(split) if split is not None else '')), dpi=500)
plt.show()